In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI
import pandas as pd
import numpy as np
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4")
from tqdm import tqdm
import pickle
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
import torch
import csv
import math



checkpoint="Salesforce/codet5p-110m-embedding"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to("cuda")
def get_code_embedding(code_snippet):
    """
    Generates embeddings for a given code snippet using a pre-trained model.

    Parameters:
    - code_snippet (str): The code for which embeddings are to be generated.
    - checkpoint (str): The model checkpoint to be used for embedding. Default is Salesforce/codet5p-110m-embedding.
    - device (str): Device to run the model on, either 'cuda' for GPU or 'cpu' for CPU. Default is 'cuda'.

    Returns:
    - torch.Tensor: Embedding tensor for the input code.
    """
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to("cuda")
    with torch.no_grad():
        embedding = model(inputs)[0]
    
    return embedding.cpu()

/u1/hfaheem/.conda/envs/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
file_path = "/u1/hfaheem/DLAndroidArtifact/my-paths-code/output/week1/output1/android_services_methods.parquet"
df = pd.read_parquet(file_path)
df.head()


,service_name,class,method,depths,access control level,json_answer
325,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserInfo(I)Landroid/content/pm/UserInfo;,"[{'depth': 0, 'java_code': 'depth : 0 public U...",SYS_OR_SIG,None
326,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserBadgeResId(I)I,"[{'depth': 0, 'java_code': 'depth : 0 public i...",SYS_OR_SIG,{'Sinks': [['UserTypeDetails v2 = p0.getUserTy...
327,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,isUserSwitcherEnabled(I)Z,"[{'depth': 0, 'java_code': 'depth : 0 public b...",SYS_OR_SIG,{'Sinks': [['int v4 = Settings$Global.getInt(v...
328,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getProfiles(IZ)Ljava/util/List;,"[{'depth': 0, 'java_code': 'depth : 0 public L...",SYS_OR_SIG,{'Sinks': [['StringBuilder v1 = new StringBuil...
329,Lcom.android.server.pm.UserManagerService,Lcom/android/server/pm/UserManagerService,getUserRestrictionSources(Ljava/lang/String;I)...,"[{'depth': 0, 'java_code': 'depth : 0 public L...",SYS_OR_SIG,{'Sinks': [['boolean v4 = p0.hasBaseUserRestri...


In [17]:
def process_json_answer(json_answer, n=2):
    global counter
    all = []
    try:
        arrays = json_answer['Sinks']
        for i, array in enumerate(arrays, 1):
            if i > n:  # Limit the number of joins to `n`
                break
            joined = '\n'.join(array)
            all.append(joined)
    except:    
        return []
    counter += 1
    return all

counter = 0
# Step 2: Apply to create a new column for joined json_answer
df['sink_code'] = df['json_answer'].apply(process_json_answer)

# display the first row first column of the DataFrame
print(f"total rows = {len(df)}")
print(f"row with valid json  = {counter}")
df.head()
df.iloc[0]["sink_code"]

total rows = 85
row with valid json  = 82


[]

In [18]:
df["embeddings"] = None
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    code_snippets = row["sink_code"]
    embeddings = []
    for each in code_snippets:
        code_embedding = get_code_embedding(each)
        embeddings.append(code_embedding)
    df.at[index, "embeddings"] = embeddings


Processing Rows: 100%|██████████| 85/85 [00:01<00:00, 70.09it/s]


Get the most similar code(prolly ignore)

In [ ]:

similarities = {}


for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
    ep1_id = row1["method"]  # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    closest_ep = None
    max_similarity = -1
    similar_sink_pair = None
    
    for index2, row2 in df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["method"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"]  # list of code snippets for the EP2
        
        # Compute pairwise similarities between all embeddings of EP1 and EP2
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > max_similarity:
                    max_similarity = sim
                    closest_ep = ep2_id
                    similar_sink_pair = (ep1_sink_code[i], ep2_sink_code[j])  
    
    similarities[ep1_id] = {
        "closest_ep": closest_ep,
        "max_similarity": max_similarity,
        "similar_sink_pair": similar_sink_pair
    }
    


Computing Similarities:   0%|          | 0/85 [00:00<?, ?it/s]/tmp/ipykernel_20623/526392682.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 85/85 [00:01<00:00, 70.31it/s]


In [ ]:
for ep, details in similarities.items():
    if details['similar_sink_pair'] is None or details['closest_ep'] is None:
        print(f"EP {ep} has no similar EPs.")
    else:
        print(f"EP {ep} is most similar to EP {details['closest_ep']} with similarity {details['max_similarity']:.4f}")
        print(f"Similar sink codes:\nEP {ep}: \n{details['similar_sink_pair'][0]}\nEP \n{details['closest_ep']}: \n{details['similar_sink_pair'][1]}")
    print("\n\n")

Get >0.8 similar

In [20]:
similarities = {}

for index1, row1 in tqdm(df.iterrows(), total=len(df), desc="Computing Similarities"):
    ep1_id = row1["method"] # name of the EP1
    ep1_embeddings = row1["embeddings"] # list of embeddings for the EP1
    ep1_sink_code = row1["sink_code"] # list of code snippets for the EP1
    
    similar_sink_pairs = []  # List to store all similar sink code pairs with similarity > 0.8
    
    for index2, row2 in df.iterrows():
        if index1 == index2:
            continue  
        
        ep2_id = row2["method"] # name of the EP2
        ep2_embeddings = row2["embeddings"] # list of embeddings for the EP2
        ep2_sink_code = row2["sink_code"] # list of code snippets for the EP2  
        
        # Compute similarities
        for i, emb1 in enumerate(ep1_embeddings):
            for j, emb2 in enumerate(ep2_embeddings):
                sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
                if sim > 0.8:  # Only consider pairs with similarity > 0.8
                    similar_sink_pairs.append({
                        "similarity": sim,
                        "ep1_code": ep1_sink_code[i],
                        "ep2_id": ep2_id,
                        "ep2_code": ep2_sink_code[j]
                    })
    
    # Store all similar pairs for the current EP
    similarities[ep1_id] = similar_sink_pairs

Computing Similarities:   0%|          | 0/85 [00:00<?, ?it/s]/tmp/ipykernel_20623/1097580516.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sim = torch.nn.functional.cosine_similarity(torch.tensor(emb1), torch.tensor(emb2), dim=0).item()
Computing Similarities: 100%|██████████| 85/85 [00:01<00:00, 69.65it/s]


In [ ]:
# Display results
for ep, similar_pairs in similarities.items():
    if not similar_pairs:
        print(f"EP {ep} has no similar EPs with similarity > 0.8.")
    else:
        print(f"EP {ep} has the following similar sink code pairs with similarity > 0.8:")
        for pair in similar_pairs:
            print(f"  - Similarity: {pair['similarity']:.4f}")
            print(f"    EP {ep}: \n{pair['ep1_code']}")
            print(f"    EP {pair['ep2_id']}: \n{pair['ep2_code']}")
            print()
    print("\n\n")

Write >0.8 similar and unique with top similarity

In [21]:
CSV_FILE = './similarities'

# Write the data to a CSV file
with open(CSV_FILE+ "_allcode" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

    # Iterate through the data and write each entry
    for ep, similar_pairs in similarities.items():
        if similar_pairs:
            for pair in similar_pairs:
                writer.writerow([
                    ep,
                    pair['ep2_id'],
                    pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['similarity']
                ])
        else:
            writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

# Writing the data to a CSV file
with open(CSV_FILE + "_score" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["EP", "EP2_ID", "Max_Similarity"])
    
    for ep, similar_pairs in similarities.items():
        if not similar_pairs:
            writer.writerow([ep, "No similar EPs", ""])
        else:
            max_similarity_per_ep2 = {}
            for pair in similar_pairs:
                ep2_id = pair['ep2_id']
                similarity = pair['similarity']
                if ep2_id not in max_similarity_per_ep2:
                    max_similarity_per_ep2[ep2_id] = similarity
                else:
                    max_similarity_per_ep2[ep2_id] = max(max_similarity_per_ep2[ep2_id], similarity)
            
            # Sorting the EP2 IDs by similarity in descending order
            sorted_ep2_ids = sorted(max_similarity_per_ep2.items(), key=lambda x: x[1], reverse=True)
            
            # Writing sorted EP2 IDs and their max similarity
            for ep2_id, max_similarity in sorted_ep2_ids:
                writer.writerow([ep, ep2_id, f"{max_similarity:.4f}"])



with open(CSV_FILE + "_top2code" + ".csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write header
    writer.writerow(["EP1_ID", "EP2_ID", "EP1_Code", "EP2_Code", "Similarity"])

    # Iterate through the data and write only the top 2 most similar pairs
    for ep, similar_pairs in similarities.items():
        
        if similar_pairs:
            # Sort the pairs by similarity in descending order
            top_pairs = sorted(similar_pairs, key=lambda x: x["similarity"], reverse=True)[:2]
            for pair in top_pairs:
                writer.writerow([
                    ep,
                    pair['ep2_id'],
                    pair['ep1_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['ep2_code'].replace("\n", " "),  # Replace newlines for better CSV readability
                    pair['similarity']
                ])
        else:
            writer.writerow([ep, "No similar EPs with similarity > 0.8", "", "", ""])

print(f"Data has been written to {CSV_FILE}")

Data has been written to ./similarities
